In [14]:
# 卷积神经网络
import tensorflow as tf
import numpy as np

def read_and_decode(filename):
    #根据文件名生成一个队列
    filename_queue = tf.train.string_input_producer([filename]) #选择要读取的文件的名字，用 tf.train.string_input_producer 函数来生成文件名队列，这个函数可以设置shuffle = Ture，来打乱队列，可以设置epoch = x，过x遍训练数据。
    reader = tf.TFRecordReader() #文件读取器
    _, serialized_example = reader.read(filename_queue)  #返回文件名和文件
    features = tf.parse_single_example(serialized_example, #通过解析器tf.parse_single_example解析
                                       features={
                                       'label': tf.FixedLenFeature([], tf.int64),
                                       'img_raw' : tf.FixedLenFeature([], tf.string),
                                       })
    image = tf.decode_raw(features['img_raw'], tf.uint8) #用解码器 tf.decode_raw 解码。
    image = tf.cast(image, dtype='float32')*(1/255)  # 归一化处理
    image = tf.reshape(image, [48, 24, 3]) # 恢复数据形状
    image = tf.split(image, 3, 2)[0]
    label = tf.cast(features['label'], tf.int32)
    return image, label

# 定义好初始化函数以便重复使用。给权重制造一些随机噪声来打破完全对称，使用截断的正态分布，标准差设为0.1，
# 同时因为使用relu，也给偏执增加一些小的正值(0.1)用来避免死亡节点(dead neurons)
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME') # 参数分别指定了卷积核的尺寸、多少个channel、filter的个数即产生特征图的个数

# 2x2最大池化，即将一个2x2的像素块降为1x1的像素。最大池化会保留原始像素块中灰度值最高的那一个像素，即保留最显著的特征。
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


n_input  = 1152 # 24*48的灰度图，像素个数1152
# 在设计网络结构前，先定义输入的placeholder，x是特征，y是真实的label
x = tf.placeholder(tf.float32, [None,n_input]) 
y = tf.placeholder(tf.int64, [None])
x_image = tf.reshape(x, [-1, 48, 24, 1]) # 对图像做预处理，将1D的输入向量转为2D的图片结构，即1*1152到24*48的结构,-1代表样本数量不固定，1代表颜色通道数量

#------------------------------------------------
#定义CNN结构
#------------------------------------------------

# 定义第一个卷积层，使用前面写好的函数进行参数初始化，包括weight和bias
W_conv1 = weight_variable([3, 3, 1, 32])  # 前两个维度代表了过滤器的尺寸，第三个维度表示当前曾的深度，第四个维度表示过滤器的深度。
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
#-1*48*24*32
#池化
h_pool1 = max_pool_2x2(h_conv1)
#-1*24*12*32

# 定义第二个卷积层
W_conv2 = weight_variable([3, 3, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
#-1*24*12*64
#池化
h_pool2 = max_pool_2x2(h_conv2)
#-1*12*6*96

#定义第三个卷积层
W_conv3 = weight_variable([3, 3, 64, 96])
b_conv3 = bias_variable([96])
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)

#-1*12*6*96
#池化
h_pool3 = max_pool_2x2(h_conv3)
#-1*6*3*96

# 全连接层
W_fc1 = weight_variable([6*3*96, 512])  ###++仔细算下卷积-池化-卷积-池化后得到的shape
b_fc1 = bias_variable([512])
h_pool2_flat = tf.reshape(h_pool3, [-1, 6*3*96])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# 为了减轻过拟合，使用Dropout层
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Dropout层输出连接一个Softmax层,得到最后的概率输出
W_fc2 = weight_variable([512, 34])
b_fc2 = bias_variable([34])

pred_not_softmax=tf.matmul(h_fc1_drop, W_fc2) + b_fc2

cross_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=pred_not_softmax,labels=y)
cost=tf.reduce_mean(cross_entropy)

#输出层和交叉熵的定义
out_op=tf.train.AdamOptimizer().minimize(cost)

#预测正确率
corr = tf.equal(tf.argmax(pred_not_softmax, 1), y) # 对比预测值的索引和真实label的索引是否一样，一样返回True，不一样返回False
accuracy = tf.reduce_mean(tf.cast(corr, tf.float32))


#读取训练集数据
image, label = read_and_decode("train.tfrecords")
batch_size=20
image_batch, label_batch = tf.train.shuffle_batch([image,label], batch_size=20, capacity=2000, min_after_dequeue=1900, num_threads=2)   #每进行一次迭代选择20个样本

#读取验证集数据
vimage, vlabel = read_and_decode("validation.tfrecords")
vimage_batch, vlabel_batch = tf.train.shuffle_batch([vimage,vlabel], batch_size=20, capacity=2000, min_after_dequeue=1900, num_threads=2)   #每进行一次迭代选择100个样本

# 初始化所有参数
init = tf.global_variables_initializer()
a = 0
a1 = 0
b = 0
with tf.Session() as sess:
    sess.run(init)
    coord = tf.train.Coordinator() #创建一个协调器，管理线程
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)   #启动QueueRunner, 此时文件名队列已经进队
    for i in range(2000):  # 规定出队数量
        img, label = sess.run([image_batch, label_batch])
        img = np.reshape(img, [-1,1152])
        label = np.reshape(label,[-1])
        out_op.run(feed_dict={x:img, y:label, keep_prob:0.7})
        
        vimg, vlabel = sess.run([vimage_batch, vlabel_batch])
        vimg = np.reshape(vimg, [-1,1152])
        vlabel = np.reshape(vlabel,[-1])
        out_op.run(feed_dict={x:vimg, y:vlabel, keep_prob:0.7})
        if i % 100 == 0: # 每100次训练，对准确率进行一次测试
            train_accuracy = accuracy.eval(feed_dict={x:img, y:label, keep_prob:1.0})
            validation_accuracy = accuracy.eval(feed_dict = {x:vimg, y:vlabel, keep_prob:1.0})
            print("step: %d  TRAIN ACCURACY: %.3f  VALIDATION ACCURACY: %.3f" % (i, train_accuracy, validation_accuracy))
            a = a + train_accuracy
            a1 += validation_accuracy
            b += 1
    print('TRAIN ACCURACY: %.3f  VALIDATION ACCURACY: %.3f' %(a/b, a1/b))
    coord.request_stop()
    coord.join(threads)

step: 0  TRAIN ACCURACY: 0.200  VALIDATION ACCURACY: 0.200
step: 100  TRAIN ACCURACY: 1.000  VALIDATION ACCURACY: 0.950
step: 200  TRAIN ACCURACY: 1.000  VALIDATION ACCURACY: 1.000
step: 300  TRAIN ACCURACY: 1.000  VALIDATION ACCURACY: 1.000
step: 400  TRAIN ACCURACY: 1.000  VALIDATION ACCURACY: 0.900
step: 500  TRAIN ACCURACY: 1.000  VALIDATION ACCURACY: 1.000
step: 600  TRAIN ACCURACY: 1.000  VALIDATION ACCURACY: 1.000
step: 700  TRAIN ACCURACY: 0.950  VALIDATION ACCURACY: 1.000
step: 800  TRAIN ACCURACY: 1.000  VALIDATION ACCURACY: 1.000
step: 900  TRAIN ACCURACY: 1.000  VALIDATION ACCURACY: 1.000
step: 1000  TRAIN ACCURACY: 1.000  VALIDATION ACCURACY: 1.000
step: 1100  TRAIN ACCURACY: 1.000  VALIDATION ACCURACY: 0.950
step: 1200  TRAIN ACCURACY: 1.000  VALIDATION ACCURACY: 1.000
step: 1300  TRAIN ACCURACY: 1.000  VALIDATION ACCURACY: 1.000
step: 1400  TRAIN ACCURACY: 1.000  VALIDATION ACCURACY: 1.000
step: 1500  TRAIN ACCURACY: 1.000  VALIDATION ACCURACY: 1.000
step: 1600  TRAIN AC